<h1>
Problema das Rainhas
</h1>

Em um tabuleiro 𝑛×𝑛, uma rainha é colocada em um quadrado, irá dominar todos os quadrados que estiverem na mesma linha, coluna e diagonais. A ideia por trás deste probelma é achar a quantidade mínima de rainhas necessárias para dominar o tabuleiro inteiro. Dominar, neste problema, significa cobrir todos os quadrados possíveis sendo atacados por rainhas incluindo aqueles onde as rainhas se encontram.